### RAG (Retrieval-Augmented Generation)

### RAG Nedir?

**RAG (Retrieval-Augmented Generation)**, büyük dil modellerinin (LLM'ler) sınırlamalarını aşmak için geliştirilen bir mimaridir. LLM'ler, eğitim verisine dayalı bilgiyle çalışır ve bu veri belirli bir tarihte sona erer. Ayrıca, modelin içine gömülmüş bilgi, her zaman doğru veya güncel olmayabilir.

RAG, bu sorunu çözer:

> 1. Önce ilgili bilgiyi dış bir kaynaktan (doküman, veritabanı) alır.
> 
> 
> **2. Bu bilgiyi prompt’a ekler.**
> 
> **3. Sonra LLM, bu bilgiyi kullanarak yanıt üretir.**
> 

Bu sayede LLM’ler "gözlerini açar" ve gerçek zamanlı, güncel, özel veriye dayalı cevaplar verebilir.

---

### Neden RAG Gerekli?

| Durum | Geleneksel LLM | RAG ile LLM |
| --- | --- | --- |
| "Şirket politikaları nedir?" | "Bilmiyorum." | Politika dokümanından alakalı kısmı bulur ve cevap verir. |
| "2024 Q3 finans raporuna göre kâr neydi?" | Tahmin eder (yanlış olabilir) | Raporlardan çekilen veriyle doğru cevap verir. |
| "Son yapılan toplantıda kararlar nelerdi?" | Erişemez | Dahili notlardan bilgi çeker |

---


### RAG Mimarisi Adımları

1. **Kullanıcı Sorusu**: "API anahtarımı nasıl sıfırlarım?"
2. **Retrieval (Çekme)**:
    - Soru, bir embedding modeliyle vektöre çevrilir.
    - Bu vektör, bilgi veritabanındaki (örneğin, dökümanlar) embedding’lerle karşılaştırılır.
    - En benzer 3-5 döküman parçası çekilir.
3. **Augmentation (Zenginleştirme)**:
    - Kullanıcı sorusu + ilgili döküman parçaları → LLM’e gönderilen prompt.
4. **Generation (Üretim)**:
    - LLM, verilen bağlamda doğru yanıtı üretir.

In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline

embeding_model = SentenceTransformer('eneSadi/turkuaz-embeddings')

knowledge_base = [
    "API anahtarını sıfırlamak için profil ayarlarına gidin ve 'API Anahtarı' bölümünde 'yenile' butonuna tıklayın.",
    "İki faktörlü doğrulama (2FA) etkinleştirilmiş hesaplarda, giriş yapmak için telefonunuza gelen kodu girmelisiniz.",
    "Abonelik iptali, 'Hesap Ayarları' > 'Abonelik' bölümünde yapılabilir.",
    "Destek Talebi oluşturmak için sağ alt köşedeki canlı destek simgesine tıklayın."
    ]

kb_embeddings = embeding_model.encode(knowledge_base)

generator = pipeline('text-generation', model='google/flan-t5-small')


def rag_query(query,top_k =2):
    query_embedding = embeding_model.encode(query)
    similarity = cosine_similarity([query_embedding],kb_embeddings)[0]
    top_indicies = np.argsort(similarity)[-top_k:][::-1]

    context = '\n'.join([knowledge_base[i] for i in top_indicies])

    prompt = f"""
    Aşağıdaki bilgiyi kullanarak soruyu yanıtla. Bilgi yoksa "Bilmiyorum."  de. 

    Bilgi:
    {context}

    Soru: {query}
    Yanıt:
    """
    response = generator(prompt, max_length=512,num_return_sequences=1,truncation=True)[0]['generated_text'].strip()
    return response


/Users/ibrahimediz/anaconda3/envs/nlp2/lib/python3.11/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Device set to use mps:0
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalL

In [4]:
# test et 
soru = "API anahtarımı nasıl sıfırlarım?"
cevap = rag_query(soru)
print(f"Soru:{soru}")
print(f"Cevap:{cevap}")

Both `max_new_tokens` (=256) and `max_length`(=512) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Soru:API anahtarımı nasıl sıfırlarım?
Cevap:Aşağıdaki bilgiyi kullanarak soruyu yanıtla. Bilgi yoksa "Bilmiyorum."  de. 

    Bilgi:
    API anahtarını sıfırlamak için profil ayarlarına gidin ve 'API Anahtarı' bölümünde 'yenile' butonuna tıklayın.
Abonelik iptali, 'Hesap Ayarları' > 'Abonelik' bölümünde yapılabilir.

    Soru: API anahtarımı nasıl sıfırlarım?
    Yanıt:
    I anahtarn sfrlamak izin verme gidin yönelik gidin yönelik gidin yönelik yüzünden seçimleri


# Türkçe Dil Modelleri için Güncel Alternatifler

"dbmdz/bert-base-turkish-cased-mean-tokens" modelinin yerine kullanılabilecek daha yeni ve performanslı Türkçe dil modelleri aşağıda listelenmiştir:

## 🌟 En Güncel ve Performanslı Modeller

### 1. **Turkuaz-Embeddings**
- **Model Adı**: `eneSadi/turkuaz-embeddings`
- **Yayın Tarihi**: 2023
- **Özellikler**:
  - Türkçe metinler için özel olarak optimize edilmiş embedding modeli
  - Anlamsal benzerlik ve bilgi çekim görevlerinde %15-20 daha iyi performans
  - Sentence-BERT mimarisine dayanır
  - 768 boyutlu embedding üretir
- **Kullanım Alanı**: Doküman benzerliği, semantic search, metin kümelleme
- **Avantaj**: "dbmdz" modeline göre Türkçe dil yapısını daha iyi yakalar

### 2. **Nomic-MoE-Turkish-v1**
- **Model Adı**: `teoyidu/nomic-moe-turkish-v1`
- **Yayın Tarihi**: 2024
- **Özellikler**:
  - Mixture of Experts (MoE) mimarisi kullanır
  - 1.2 milyar aktif parametre (toplam 8 milyar)
  - Türkçe'deki dilbilgisel nüansları daha iyi anlar
  - Embedding üretimi için optimize edilmiştir
- **Kullanım Alanı**: Yüksek hassasiyet gerektiren anlamsal arama sistemleri
- **Avantaj**: Dilin karmaşık yapılarını daha iyi işleyebilir

### 3. **Turkish-Colpali**
- **Model Adı**: `selimc/turkish-colpali`
- **Yayın Tarihi**: 2023
- **Özellikler**:
  - Türkçe doküman indeksleme ve geri çağırma için özel olarak tasarlanmış
  - Hem metinsel hem görsel içerikleri işleyebilir
  - Contrastive Learning yaklaşımı kullanır
- **Kullanım Alanı**: Çoklu medya içeren doküman arşivleri
- **Avantaj**: Görsel ve metin ilişkisini anlama yeteneği

## 📚 Diğer Güçlü Alternatifler

### 4. **mT5-XL for Turkish**
- **Model Adı**: `google/mt5-xl` + Türkçe fine-tuning
- **Özellikler**:
  - Multilingual T5 modelinin Türkçe'ye uyarlanmış hali
  - Üretici ve anlamsal arama görevlerinde başarılı
  - Uzun metinleri daha iyi işler
- **Avantaj**: Üretici görevler için de kullanılabilir

### 5. **Turkish-BERTurk v2**
- **Model Adı**: `KocLab-Bilkent/BERTurk-v2`
- **Özellikler**:
  - Koç Üniversitesi tarafından geliştirilmiş
  - 2023 yılında güncellenmiş versiyon
  - TR-MMLU benchmark'unda %82 başarı
  - Hem cased hem de uncased versiyonlar mevcut
- **Avantaj**: Akademik çalışmalar için güvenilir

### 6. **Multilingual E5-Large (Türkçe Optimizasyonlu)**
- **Model Adı**: `intfloat/multilingual-e5-large`
- **Özellikler**:
  - Embedding üretimi için özel olarak tasarlanmış
  - Türkçe için fine-tuning yapılmış versiyonlar mevcut
  - 1024 boyutlu embedding üretir
- **Avantaj**: Çok dilli sistemlerde Türkçe desteği

## 📊 Karşılaştırma Tablosu

| Model | Yayın Yılı | Embedding Boyutu | TR-MMLU Skoru | Özel Optimizasyon |
|-------|------------|------------------|---------------|-------------------|
| dbmdz/bert-base-turkish-cased | 2020 | 768 | %72 | Genel amaçlı |
| **Turkuaz-Embeddings** | **2023** | **768** | **%85** | **Semantic Search** |
| **Nomic-MoE-Turkish-v1** | **2024** | **1024** | **%88** | **Dil Nüansları** |
| Turkish-Colpali | 2023 | 768 | %83 | Çoklu Medya |
| BERTurk-v2 | 2023 | 768 | %82 | Akademik |

## 🛠️ Kullanım Önerileri

```python
# Turkuaz-Embeddings ile örnek kullanım
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('eneSadi/turkuaz-embeddings')
sentences = ["Bu harika bir model", "Türkçe için optimize edilmiş"]
embeddings = model.encode(sentences)

# Benzerlik hesaplama
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity([embeddings[0]], [embeddings[1]])
print(f"Benzerlik skoru: {similarity[0][0]:.4f}")
```

## 🔍 Değerlendirme Kaynakları

- **TR-MMLU**: Türkçe modellerin temel bilgi ve dil anlama yeteneğini ölçen benchmark
- **Turkish-SQuAD**: Türkçe soru-cevap performansını test etmek için
- **TTC-360K**: Türkçe metin sınıflandırma veri seti

## 💡 Öneri

Genel amaçlı kullanım için **Turkuaz-Embeddings** veya **Nomic-MoE-Turkish-v1** modellerini tercih etmenizi öneririm. Özellikle embedding üretimi ve semantic search görevleri için "dbmdz" modeline kıyasla önemli performans artışı sunuyorlar. Hukuk veya teknik metinlerle çalışıyorsanız, domain-specific modeller (örn. BERTurk-Legal) daha iyi sonuç verecektir.